## Reproduce the results using the models trained and stored in the models folder:

In [1]:
from Utils_lstm import *

2025-05-05 09:38:40.578622: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
models_path='$HOME/Datasets/QuoraQuestionPairs/models'

In [ ]:
# Create validation and test partitions
quora_df = pd.read_csv("$HOME/Datasets/QuoraQuestionPairs/quora_data.csv")
A_df, test_df = sklearn.model_selection.train_test_split(quora_df, test_size=0.05, random_state=123)
train_df, val_df = sklearn.model_selection.train_test_split(A_df, test_size=0.05)
print('val_df.shape=',val_df.shape)
print('test_df.shape=',test_df.shape)

y_val = val_df["is_duplicate"].values
y_test = test_df["is_duplicate"].values

# cast to list taking care of nans:
q1_val =  cast_list_as_strings(list(val_df["question1"]))
q2_val =  cast_list_as_strings(list(val_df["question2"]))
q1_test  =  cast_list_as_strings(list(test_df["question1"]))
q2_test  =  cast_list_as_strings(list(test_df["question2"]))

val_df.shape= (15363, 6)
test_df.shape= (16172, 6)


### 9. Siamese LSTM

In [4]:
model_lstm = load_model(models_path, "model_lstm.pkl")
# Prepare text for LSTM
q1_train =  cast_list_as_strings(list(train_df["question1"]))
q2_train =  cast_list_as_strings(list(train_df["question2"]))


max_words = 20000
max_len = 50

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)



tokenizer = load_model(models_path, "tokenizer_tf.pkl")


X_train_q1, X_train_q2 = texts_to_padded(q1_train, q2_train, tokenizer, max_len)
X_val_q1,   X_val_q2   = texts_to_padded(q1_val, q2_val, tokenizer, max_len)
X_test_q1,  X_test_q2  = texts_to_padded(q1_test, q2_test, tokenizer, max_len)
# Evaluate LSTM
y_pred_lstm = (model_lstm.predict([X_test_q1, X_test_q2]) > 0.6).astype(int)
lstm_acc = accuracy_score(y_test, y_pred_lstm)

print(f"LSTM Accuracy: {lstm_acc}")

Model loaded from /Users/user/Documents/GitHub/nlp_deliv_1/models/model_lstm.pkl
Model loaded from /Users/user/Documents/GitHub/nlp_deliv_1/models/tokenizer_tf.pkl
506/506 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step
LSTM Accuracy: 0.8044768736087065
